In [1]:
## Memorializing this as the easy way to lookup accounts


In [6]:
"""
Main Entry Point
"""
from pathlib import Path
from typing import List
import argparse
from datetime import datetime

import pandas as pd # type: ignore

import src.fetch_data # type: ignore
import src.core_transform # type: ignore
import cdutils.pkey_sqlite # type: ignore
import cdutils.hhnbr # type: ignore


data = src.fetch_data.fetch_data()

# # # Core transformation pipeline
raw_data = src.core_transform.main_pipeline(data)

# Raw data with pkey appended
raw_data = cdutils.pkey_sqlite.add_pkey(raw_data)

 

In [7]:
househldacct = data['househldacct'].copy()

In [8]:
raw_data = cdutils.hhnbr.add_hh_nbr(raw_data, househldacct)

In [9]:
raw_data

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,origbal,loanlimityn,Net Balance,Net Available,Net Collateral Reserve,Total Exposure,orig_ttl_loan_amt,portfolio_key,householdnbr,datelastmaint
0,2025-04-23,150200005,MONEYGRAM PAYMENT SYSTEMS INC,Treasurer's Check,0.0,None,BKCK,TRCK,ACT,0.00000,...,NaN,NaN,1.544759e+09,0.0,0.0,1.544759e+09,0.0,42125,187340.0,2020-02-26 23:28:03
1,2025-04-23,150936915,REDBROOK APARTMENTS LLC,Commercial Mortgages,65500000.0,VAR,CML,CM40,ACT,0.06569,...,65500000.0,N,2.900000e+07,0.0,0.0,2.900000e+07,65500000.0,42025,193906.0,2023-09-22 08:01:38
2,2025-04-23,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.06350,...,26000000.0,N,2.600000e+07,0.0,0.0,2.600000e+07,26000000.0,80094,288455.0,2024-06-11 12:02:04
3,2025-04-23,150847394,COUNTY OF BRISTOL,Municipal Money Market,0.0,VAR,CK,CK18,ACT,0.03500,...,NaN,NaN,2.356020e+07,0.0,0.0,2.356020e+07,0.0,4335,NaN,NaT
4,2025-04-23,150200013,MONEYGRAM PAYMENT SYSTEMS INC,Money Order,0.0,None,BKCK,MO,ACT,0.00000,...,NaN,NaN,2.181384e+07,0.0,0.0,2.181384e+07,0.0,42125,187340.0,2020-02-26 23:28:13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90043,2025-04-23,26178737,"MILICH, LISA A.",eChecking (18 & over),0.0,None,CK,CK04,ACT,0.00000,...,NaN,NaN,-1.093630e+03,0.0,0.0,-1.093630e+03,0.0,37993,149217.0,2020-02-26 23:28:12
90044,2025-04-23,27047636,BRISTOL COUNTY SAVINGS BANK,Payoff Overages,0.0,None,BKCK,BTC2,ACT,0.00000,...,NaN,NaN,-2.098590e+03,0.0,0.0,-2.098590e+03,0.0,1293,255924.0,2022-09-27 22:46:27
90045,2025-04-23,27039064,BRISTOL COUNTY SAVINGS BANK,Escrow Checks Processing,0.0,None,BKCK,BTC3,ACT,0.00000,...,NaN,NaN,-7.325560e+04,0.0,0.0,-7.325560e+04,0.0,1293,255924.0,2022-09-27 22:46:27
90046,2025-04-23,150337783,BRISTOL COUNTY SAVINGS BANK,Business Checking,0.0,None,CK,CK12,ACT,0.00000,...,NaN,NaN,-1.500849e+05,0.0,0.0,-1.500849e+05,0.0,1293,255924.0,2022-09-27 22:46:27


In [10]:
import cdutils.loans.calculations

In [11]:
loan_category_df = cdutils.loans.calculations.categorize_loans(raw_data)

In [12]:
loan_category_df = loan_category_df[['acctnbr','Category']].copy()

In [13]:
df = pd.merge(raw_data, loan_category_df, on='acctnbr', how='left')

In [14]:
df

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,loanlimityn,Net Balance,Net Available,Net Collateral Reserve,Total Exposure,orig_ttl_loan_amt,portfolio_key,householdnbr,datelastmaint,Category
0,2025-04-23,150200005,MONEYGRAM PAYMENT SYSTEMS INC,Treasurer's Check,0.0,None,BKCK,TRCK,ACT,0.00000,...,NaN,1.544759e+09,0.0,0.0,1.544759e+09,0.0,42125,187340.0,2020-02-26 23:28:03,NaN
1,2025-04-23,150936915,REDBROOK APARTMENTS LLC,Commercial Mortgages,65500000.0,VAR,CML,CM40,ACT,0.06569,...,N,2.900000e+07,0.0,0.0,2.900000e+07,65500000.0,42025,193906.0,2023-09-22 08:01:38,CRE
2,2025-04-23,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.06350,...,N,2.600000e+07,0.0,0.0,2.600000e+07,26000000.0,80094,288455.0,2024-06-11 12:02:04,CRE
3,2025-04-23,150847394,COUNTY OF BRISTOL,Municipal Money Market,0.0,VAR,CK,CK18,ACT,0.03500,...,NaN,2.356020e+07,0.0,0.0,2.356020e+07,0.0,4335,NaN,NaT,NaN
4,2025-04-23,150200013,MONEYGRAM PAYMENT SYSTEMS INC,Money Order,0.0,None,BKCK,MO,ACT,0.00000,...,NaN,2.181384e+07,0.0,0.0,2.181384e+07,0.0,42125,187340.0,2020-02-26 23:28:13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90043,2025-04-23,26178737,"MILICH, LISA A.",eChecking (18 & over),0.0,None,CK,CK04,ACT,0.00000,...,NaN,-1.093630e+03,0.0,0.0,-1.093630e+03,0.0,37993,149217.0,2020-02-26 23:28:12,NaN
90044,2025-04-23,27047636,BRISTOL COUNTY SAVINGS BANK,Payoff Overages,0.0,None,BKCK,BTC2,ACT,0.00000,...,NaN,-2.098590e+03,0.0,0.0,-2.098590e+03,0.0,1293,255924.0,2022-09-27 22:46:27,NaN
90045,2025-04-23,27039064,BRISTOL COUNTY SAVINGS BANK,Escrow Checks Processing,0.0,None,BKCK,BTC3,ACT,0.00000,...,NaN,-7.325560e+04,0.0,0.0,-7.325560e+04,0.0,1293,255924.0,2022-09-27 22:46:27,NaN
90046,2025-04-23,150337783,BRISTOL COUNTY SAVINGS BANK,Business Checking,0.0,None,CK,CK12,ACT,0.00000,...,NaN,-1.500849e+05,0.0,0.0,-1.500849e+05,0.0,1293,255924.0,2022-09-27 22:46:27,NaN


In [22]:
pkey_slice = df[['acctnbr','portfolio_key']].copy()

In [23]:
pkey_slice = pkey_slice.rename(columns={'portfolio_key':'HHNU'}).copy()

In [24]:
pkey_slice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90048 entries, 0 to 90047
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   acctnbr  90048 non-null  object
 1   HHNU     90048 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ MB


In [25]:
OUTPUT_PATH = 'hhnu.csv'
pkey_slice.to_csv(OUTPUT_PATH, index=False)

In [26]:
pkey_small_chunk = pkey_slice.head(1000).copy()

In [27]:

OUTPUT_PATH = 'hhnu_first_1000.csv'
pkey_small_chunk.to_csv(OUTPUT_PATH, index=False)
